In [2]:
import numpy as np
import h5py
import sys, os

In [7]:
simba_params = np.loadtxt('CosmoAstroSeed_SIMBA.txt', skiprows=1, 
           dtype={'names': ('name', 'Om', 's8', 'Asn1', 'Aagn1', 'Asn2', 'Aagn2', 'seed'),
                  'formats': ('S4', 'f', 'f', 'f', 'f', 'f', 'f', 'i')})

In [14]:
# set the threshold to remove spurious galaxies
Nstars_thres = 20

props_lhc = []
for i_lhc in range(1000): 
    # get the name of the SUBFIND catalogues
    f_subfind = '/home/jovyan/Data/FOF_Subfind/SIMBA/LH_%i/fof_subhalo_tab_033.hdf5' % i_lhc

    # open the catalogue and read it
    f = h5py.File(f_subfind, 'r')

    Mg     = f['/Subhalo/SubhaloMassType'][:,0]*1e10 #Msun/h
    Mstar  = f['/Subhalo/SubhaloMassType'][:,4]*1e10 #Msun/h
    Mbh    = f['/Subhalo/SubhaloBHMass'][:]*1e10     #Msun/h
    Mtot   = f['/Subhalo/SubhaloMass'][:]*1e10       #Msun/h

    Vmax   = f['/Subhalo/SubhaloVmax'][:]
    Vdisp  = f['/Subhalo/SubhaloVelDisp'][:]

    Zg     = f['/Subhalo/SubhaloGasMetallicity'][:]
    Zs     = f['/Subhalo/SubhaloStarMetallicity'][:]
    SFR    = f['/Subhalo/SubhaloSFR'][:]
    J      = f['/Subhalo/SubhaloSpin'][:]
    Vel    = f['/Subhalo/SubhaloVel'][:]
    J      = np.sqrt(J[:,0]**2 + J[:,1]**2 + J[:,2]**2)
    Vel    = np.sqrt(Vel[:,0]**2 + Vel[:,1]**2 + Vel[:,2]**2)

    Rstar  = f['/Subhalo/SubhaloHalfmassRadType'][:,4]/1e3 #Mpc/h
    Rtot   = f['/Subhalo/SubhaloHalfmassRad'][:]/1e3       #Mpc/h
    Rvmax  = f['/Subhalo/SubhaloVmaxRad'][:]/1e3           #Mpc/h

    U      = f['/Subhalo/SubhaloStellarPhotometrics'][:,0]
    B      = f['/Subhalo/SubhaloStellarPhotometrics'][:,1]
    V      = f['/Subhalo/SubhaloStellarPhotometrics'][:,2]    
    K      = f['/Subhalo/SubhaloStellarPhotometrics'][:,3]
    g      = f['/Subhalo/SubhaloStellarPhotometrics'][:,4]
    r      = f['/Subhalo/SubhaloStellarPhotometrics'][:,5]
    i      = f['/Subhalo/SubhaloStellarPhotometrics'][:,6]
    z      = f['/Subhalo/SubhaloStellarPhotometrics'][:,7]

    
    Nstars = f['/Subhalo/SubhaloLenType'][:,4]
    f.close()

    # only take galaxies with more than 20 stars
    indexes = np.where(Nstars>Nstars_thres)[0]
    Ngal    = len(indexes)
        
    # compile table of 
    # Om, s8, Asn1, Aagn1, Asn2, Aagn2, Mg, Mstar, Mbh, Mtot, Vmax, Vdisp, Zg, Zs, SFR, J, Vel, Rstar, Rtot, Rvmax,
    # absmag U, B, V, K, g, r, i, z
    _props = np.array([
        np.repeat(simba_params['Om'][i_lhc], Ngal), 
        np.repeat(simba_params['s8'][i_lhc], Ngal), 
        np.repeat(simba_params['Asn1'][i_lhc], Ngal), 
        np.repeat(simba_params['Aagn1'][i_lhc], Ngal), 
        np.repeat(simba_params['Asn2'][i_lhc], Ngal), 
        np.repeat(simba_params['Aagn2'][i_lhc], Ngal),   
        Mg[indexes], 
        Mstar[indexes],
        Mbh[indexes], 
        Mtot[indexes], 
        Vmax[indexes], 
        Vdisp[indexes], 
        Zg[indexes],
        Zs[indexes], 
        SFR[indexes], 
        J[indexes], 
        Vel[indexes], 
        Rstar[indexes], 
        Rtot[indexes],
        Rvmax[indexes], 
        U[indexes], 
        B[indexes], 
        V[indexes], 
        K[indexes],         
        g[indexes],         
        r[indexes], 
        i[indexes], 
        z[indexes]])
    props_lhc.append(_props)

In [24]:
props_lhc = np.concatenate(props_lhc, axis=1)

In [32]:
f = h5py.File('simba.snap33.subfind.galaxies.LHC.hdf5', 'w')
f.create_dataset('props', data=props_lhc)
f.close()